# Iterative Model Development Steps with Application to Airlines Dataset
- Steps are outlined in https://goo.gl/A7P4vX
- Link to airlines [dataset](https://github.com/h2oai/h2o-2/wiki/Hacking-Airline-DataSet-with-H2O)

In [1]:
from collections import Counter
import inspect
from joblib import dump, load
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score

In [2]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 8

## Step 1: Understand Different Modeling Approaches
- Model development is an art and science -- you may have done these steps differently. 
- Please let us know what you would have done!

## Step 2: Understand Business Use Case
Proposed use case -- there may be (many) others:
- Client: Airline
- Statement of Problem: Airline has to compensate passangers if flight was delayed by 2+ hours or if flight arrived 3+ hours later.
- Question: Are there (any) aspects of delay that could have been prevented?

- Production environment:
  - Jupyter notebook (for POC)
  - Running Python 3.7
  - requirements.txt

- Outcome variable: (arrival delay of 3+ hours or departure delay of 2+ hours) or not, per https://upgradedpoints.com/flight-delay-cancelation-compensation

## Step 3: Get Access to Data

### Step 3-a: Read-in Data

In [3]:
file_name = "https://s3.amazonaws.com/h2o-airlines-unpacked/year2012.csv"
df = pd.read_csv(filepath_or_buffer=file_name,
                 encoding='latin-1')
# df = pd.read_csv("2012.csv")

### Step 3-b: EDA of Data

In [4]:
df.shape

(6096762, 31)

In [5]:
Counter(df['Month'])

Counter({1: 486133,
         2: 464826,
         3: 521628,
         4: 505218,
         5: 518423,
         6: 526933,
         7: 545131,
         8: 540793,
         9: 490199,
         10: 515254,
         11: 488006,
         12: 494218})

In [6]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
0,2012,1,1,7,855.0,900.0,1142.0,1225.0,AA,1,N325AA,347.0,385.0,330.0,-43.0,-5.0,JFK,LAX,2475,4.0,13.0,0,NaN,0,0,0,0,0,0,NO,NO
1,2012,1,2,1,921.0,900.0,1210.0,1225.0,AA,1,N319AA,349.0,385.0,325.0,-15.0,21.0,JFK,LAX,2475,11.0,13.0,0,NaN,0,0,0,0,0,0,NO,YES
2,2012,1,3,2,931.0,900.0,1224.0,1225.0,AA,1,N323AA,353.0,385.0,319.0,-1.0,31.0,JFK,LAX,2475,22.0,12.0,0,NaN,0,0,0,0,0,0,NO,YES
3,2012,1,4,3,904.0,900.0,1151.0,1225.0,AA,1,N320AA,347.0,385.0,309.0,-34.0,4.0,JFK,LAX,2475,20.0,18.0,0,NaN,0,0,0,0,0,0,NO,YES
4,2012,1,5,4,858.0,900.0,1142.0,1225.0,AA,1,N338AA,344.0,385.0,306.0,-43.0,-2.0,JFK,LAX,2475,22.0,16.0,0,NaN,0,0,0,0,0,0,NO,NO


In [7]:
min(df['DepTime']), max(df['DepTime'])

(1.0, 2400.0)

In [8]:
min(df['ArrTime']), max(df['ArrTime'])

(1.0, 2400.0)

In [9]:
Counter(df['UniqueCarrier'])

Counter({'AA': 525220,
         'AS': 147569,
         'B6': 229056,
         'DL': 726879,
         'EV': 740855,
         'F9': 79255,
         'FL': 218162,
         'HA': 74109,
         'MQ': 473140,
         'OO': 617756,
         'UA': 531245,
         'US': 404263,
         'VX': 54742,
         'WN': 1140535,
         'YV': 133976})

## Step 4: To come a little later...

## Step 5: Feature Engineering for Baseline Model (v0)

What potential features would we want to include in model?

What features should we **not consider** for inclusion into model?

### Step 5-a: Create an Outcome Variable

In [10]:
# Similar to function introduced in Class 2:

def delays_requiring_compensation(arrival_delay, departure_delay):
    """Fcn to return if arrival and/or departure delay resulted in passenger
       compensation.
       
       Arguments:
           - arrival_delay:   delay in minutes
           - departure_delay: delay in minutes
       
       Returns:
           - number of delays (arrival and or departure) that were delayed
             so long that passenger got compensated
    """
    count = 0
    if (arrival_delay/60.0 >= 3) | (departure_delay/60.0 >= 2):
        # If arrival delay is 3+ hours, or if departure delay is 2+ hours:
        count += 1
    return count

In [11]:
df['compensated_delays'] = df[['ArrDelay', 'DepDelay']].apply(
    lambda row: delays_requiring_compensation(row[0], row[1]),
    axis=1)
df[['ArrDelay', 'DepDelay', 'compensated_delays']].head()

,ArrDelay,DepDelay,compensated_delays
0,-43.0,-5.0,0
1,-15.0,21.0,0
2,-1.0,31.0,0
3,-34.0,4.0,0
4,-43.0,-2.0,0


In [12]:
Counter(df['compensated_delays'])

Counter({0: 5995130, 1: 101632})

Let's stop and think about this number... What are client implications?

### Step 5-b: Create a Time-of-Day Variable
Per [documentation](http://stat-computing.org/dataexpo/2009/the-data.html) and EDA, time of day is recorded in hhmm.

In [13]:
# Recall:
min(df['DepTime']), max(df['DepTime'])

(1.0, 2400.0)

How is departure delay recorded?

How would you convert this field to a time-of-day?

In [14]:
print(df['DepTime'][0])
str(int(df['DepTime'][0])).zfill(4)

855.0


'0855'

In [15]:
print(min(df['DepTime']))
str(int(min(df['DepTime']))).zfill(4)

1.0


'0001'

In [16]:
# Before processing all the values, assign missing values to own category:
df['DepTime'] = df['DepTime'].fillna(9999.0)

In [60]:
df['Dep_Hour'] = df['DepTime'].apply(lambda x:
                                     int(
                                         str(int(x)).zfill(4)[0:2]
                                     ))

What would be a good comment to the code above?

In [61]:
df['Dep_Hour'].value_counts(sort=False)

0      14878
1       6135
2       1630
3        738
       ...  
22    119418
23     42057
24       293
99     75723
Name: Dep_Hour, Length: 26, dtype: int64

Does anything look weird?

In [73]:
# Convert TOD for flights that departed at 24 hours to departing at TOD=0 hours:
row_index = np.where(df['Dep_Hour'] == 24)[0].tolist()
col_index = np.where(df.columns == 'Dep_Hour')[0].tolist()[0]
df.iloc[row_index, col_index] = 0

In [72]:
df['Dep_Hour'].value_counts(sort=False)

0      15171
1       6135
2       1630
3        738
       ...  
21    200855
22    119418
23     42057
99     75723
Name: Dep_Hour, Length: 25, dtype: int64

### Step 3-c: Create Indicator Variables from Features for Use with Sklearn
Features:
- Month
- Day of Week
- Time of Day

In [21]:
features_tod = pd.get_dummies(df['Dep_Hour'], drop_first=True, prefix="tod_")

In [22]:
features_tod.head()

,tod__1,tod__2,tod__3,tod__4,tod__5,tod__6,tod__7,tod__8,tod__9,tod__10,tod__11,tod__12,tod__13,tod__14,tod__15,tod__16,tod__17,tod__18,tod__19,tod__20,tod__21,tod__22,tod__23,tod__99
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
features_month = pd.get_dummies(df['Month'], drop_first=True, prefix="mo_")

In [24]:
features_dow = pd.get_dummies(df['DayOfWeek'], drop_first=True, prefix="dow_")

In [25]:
features = pd.concat([features_tod, features_month, features_dow],
                     axis=1,
                     join='inner')

In [26]:
features.columns

Index(['tod__1', 'tod__2', 'tod__3', 'tod__4', 'tod__5', 'tod__6', 'tod__7',
       'tod__8', 'tod__9', 'tod__10', 'tod__11', 'tod__12', 'tod__13',
       'tod__14', 'tod__15', 'tod__16', 'tod__17', 'tod__18', 'tod__19',
       'tod__20', 'tod__21', 'tod__22', 'tod__23', 'tod__99', 'mo__2', 'mo__3',
       'mo__4', 'mo__5', 'mo__6', 'mo__7', 'mo__8', 'mo__9', 'mo__10',
       'mo__11', 'mo__12', 'dow__2', 'dow__3', 'dow__4', 'dow__5', 'dow__6',
       'dow__7'],
      dtype='object')

What is our baseline Month, DOW and TOD reference point?

In [27]:
dataset = pd.concat([features, df['compensated_delays']],
                     axis=1)

In [28]:
dataset.shape

(6096762, 42)

## Step 4: Determine Data Splits
What are some data splits that you would propose?


In [29]:
df_tmp, df_test = train_test_split(dataset,
                                   test_size=0.25,
                                   random_state=2019,
                                   stratify=dataset['compensated_delays'])

In [30]:
df_train, df_valid = train_test_split(df_tmp,
                                      test_size=0.25,
                                      random_state=2019,
                                      stratify=df_tmp['compensated_delays'])

In [31]:
df_train['compensated_delays'].value_counts(sort=False)

0    3372260
1      57168
Name: compensated_delays, dtype: int64

In [32]:
df_valid['compensated_delays'].value_counts(sort=False)

0    1124087
1      19056
Name: compensated_delays, dtype: int64

In [33]:
df_test['compensated_delays'].value_counts(sort=False)

0    1498783
1      25408
Name: compensated_delays, dtype: int64

## Step 6: Estimate a Baseline Model (v0)

In [34]:
y = df_train['compensated_delays']
X = df_train.drop(columns=['compensated_delays'])

In [35]:
inspect.signature(LogisticRegression)

<Signature (penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='warn', max_iter=100, multi_class='warn', verbose=0, warm_start=False, n_jobs=None)>

In [36]:
est_model = LogisticRegression(penalty="l2",
                               C=0.5,
                               fit_intercept=True,
                               class_weight='balanced',
                               random_state=2019,
                               max_iter=10000,
                               solver='lbfgs')

In [37]:
est_model

LogisticRegression(C=0.5, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=10000,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=2019,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [38]:
est_model.fit(X, y)


In [39]:
# Save model, per: https://scikit-learn.org/stable/modules/model_persistence.html
dump(est_model, 'logistic.joblib')

## Load saved model:
# est_model = load('logistic.joblib') 


Remarks:
- Aside: creating a filename that includes a [timestamp](https://stackoverflow.com/questions/10607688/how-to-create-a-file-name-with-the-current-date-time-in-python)
- Aside: If interested in seeing more `R`-like output for Logistic Regression, use library `statsmodels`
- Logistic Regression with [Cross-Validation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html)
- (In general) Cross-Validation with sklearn: [approach 1](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and [approach 2](https://scikit-learn.org/0.16/modules/generated/sklearn.grid_search.GridSearchCV.html)

## Step 7: Interpret Results

In [40]:
# DataFrame of feature coefficients:
coefficients = [round(x, 2) for x in est_model.coef_.tolist()[0]]
coef_df = pd.concat([pd.DataFrame(X.columns),
                     pd.DataFrame(coefficients)],
                    axis = 1)
coef_df.columns = ["feature_name", "coef_est"]

# DataFrame of intercept coefficient:
intercept_df = pd.DataFrame(["intercept", est_model.intercept_.tolist()[0]]).T
intercept_df.columns = ["feature_name", "coef_est"]

# Combined DataFrame:
coef_df = coef_df.append(intercept_df)
coef_df.head()

,feature_name,coef_est
0,tod__1,0.12
1,tod__2,0.22
2,tod__3,-0.09
3,tod__4,-3.26
4,tod__5,-6.82


In [41]:
# Compute odds:
coef_df["odds"] = [round(np.exp(x), 2) for x in coef_df['coef_est']]

In [42]:
# Compute probabilities, which need intercept:
intercept = intercept_df['coef_est'].values.tolist()[0]
intercept

2.730757870586944

In [43]:
def inverse_logit(intercept, coefficient):
    """Fcn to help calculate probability associated with flight delay,
       given our variable.
    """
    if coefficient == intercept:
        # Make sure we don't double-count the intercept:
        coefficient = 0
    inv_logit = np.exp(intercept + coefficient)/(1+np.exp(intercept + coefficient))
    return round(inv_logit, 2)
                           
coef_df["prob_delay"] = [inverse_logit(intercept, x) for x in coef_df['coef_est']]


In [44]:
coef_df.T


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,0
feature_name,tod__1,tod__2,tod__3,tod__4,tod__5,tod__6,tod__7,tod__8,tod__9,tod__10,tod__11,tod__12,tod__13,tod__14,tod__15,tod__16,tod__17,tod__18,tod__19,tod__20,tod__21,tod__22,tod__23,tod__99,mo__2,mo__3,mo__4,mo__5,mo__6,mo__7,mo__8,mo__9,mo__10,mo__11,mo__12,dow__2,dow__3,dow__4,dow__5,dow__6,dow__7,intercept
coef_est,0.12,0.22,-0.09,-3.26,-6.82,-6.71,-5.71,-4.26,-3.65,-3.38,-3.27,-3.09,-3,-2.97,-2.76,-2.71,-2.69,-2.47,-2.34,-1.96,-1.71,-1.28,-0.51,-9.74,-0.3,0.11,-0.29,0.03,0.23,0.59,0.36,-0.02,0.03,0,0.4,-0.26,-0.15,-0.05,0.06,-0.19,-0.17,2.73076
odds,1.13,1.25,0.91,0.04,0,0,0,0.01,0.03,0.03,0.04,0.05,0.05,0.05,0.06,0.07,0.07,0.08,0.1,0.14,0.18,0.28,0.6,0,0.74,1.12,0.75,1.03,1.26,1.8,1.43,0.98,1.03,1,1.49,0.77,0.86,0.95,1.06,0.83,0.84,15.34
prob_delay,0.95,0.95,0.93,0.37,0.02,0.02,0.05,0.18,0.29,0.34,0.37,0.41,0.43,0.44,0.49,0.51,0.51,0.56,0.6,0.68,0.74,0.81,0.9,0,0.92,0.94,0.92,0.94,0.95,0.97,0.96,0.94,0.94,0.94,0.96,0.92,0.93,0.94,0.94,0.93,0.93,0.94


How would you interpret the coefficients?

Did we have a good reference point for comparison (of coefficients) for Month, DOW and TOD?

What is a better reference point?

Can be interpret the meaning of the TOD=-99?

In [45]:
# Duplicated to make it easier for interpretations:
coef_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,0
feature_name,tod__1,tod__2,tod__3,tod__4,tod__5,tod__6,tod__7,tod__8,tod__9,tod__10,tod__11,tod__12,tod__13,tod__14,tod__15,tod__16,tod__17,tod__18,tod__19,tod__20,tod__21,tod__22,tod__23,tod__99,mo__2,mo__3,mo__4,mo__5,mo__6,mo__7,mo__8,mo__9,mo__10,mo__11,mo__12,dow__2,dow__3,dow__4,dow__5,dow__6,dow__7,intercept
coef_est,0.12,0.22,-0.09,-3.26,-6.82,-6.71,-5.71,-4.26,-3.65,-3.38,-3.27,-3.09,-3,-2.97,-2.76,-2.71,-2.69,-2.47,-2.34,-1.96,-1.71,-1.28,-0.51,-9.74,-0.3,0.11,-0.29,0.03,0.23,0.59,0.36,-0.02,0.03,0,0.4,-0.26,-0.15,-0.05,0.06,-0.19,-0.17,2.73076
odds,1.13,1.25,0.91,0.04,0,0,0,0.01,0.03,0.03,0.04,0.05,0.05,0.05,0.06,0.07,0.07,0.08,0.1,0.14,0.18,0.28,0.6,0,0.74,1.12,0.75,1.03,1.26,1.8,1.43,0.98,1.03,1,1.49,0.77,0.86,0.95,1.06,0.83,0.84,15.34
prob_delay,0.95,0.95,0.93,0.37,0.02,0.02,0.05,0.18,0.29,0.34,0.37,0.41,0.43,0.44,0.49,0.51,0.51,0.56,0.6,0.68,0.74,0.81,0.9,0,0.92,0.94,0.92,0.94,0.95,0.97,0.96,0.94,0.94,0.94,0.96,0.92,0.93,0.94,0.94,0.93,0.93,0.94


**Interpretations:**
Relative to midnight flights leaving on Mondays in January (e.g. intercept = baseline),
- "Red eye" departures tends to increase odds of compensated delays for passengers
- Flights over:
  - Spring break or
  - in summer or
  - winter holidays
tend to increase odds of compensated delays for passengers
- Friday departures also tend to increase odds of compensated delays for passengers

**Takeaways/Recommendations to your airline client?**


**As a passenger, what's the best time to travel?**

## Step 8: Evaluate Performance

### Step 8a: Evaluate Performance on In-Sample Data
Evaluate performance on in-sample data, to see wat the "best-possible" performance is, on data that the model's seen.

Aside: list available metrics available in `sklearn` [here](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

In [46]:
y_pred_train = est_model.predict(X)
y_pred_train[0:5]

array([0, 1, 0, 0, 0])

In [47]:
y_pred_train_probs = pd.DataFrame(est_model.predict_proba(X))
y_pred_train_probs.head()

,0,1
0,0.979545,0.020455
1,0.435304,0.564696
2,0.675102,0.324898
3,0.977102,0.022898
4,0.695111,0.304889


#### Evaluate Performance via Confusion Matrix

In [48]:
inspect.signature(confusion_matrix)

<Signature (y_true, y_pred, labels=None, sample_weight=None)>

In [49]:
confusion_matrix(y_true=y,
                 y_pred=y_pred_train)

array([[2291437, 1080823],
       [  17863,   39305]])

What does this confusion matrix tell us about our model?

#### Evaluating Performance with AUC ROC

In [50]:
inspect.signature(roc_auc_score)

<Signature (y_true, y_score, average='macro', sample_weight=None, max_fpr=None)>

In [51]:
# Per documentation
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score
# y_score can contain "probability estimates of the positive class":
roc_auc_score(y_true=y,
              y_score=y_pred_train_probs.iloc[:, 1])

0.7651031323687287

#### Evaluating Performance via F1

In [52]:
f1_score(y_true=y,
         y_pred=y_pred_train)

0.06677165300824942

Is our F1-score surprising?


![F1-score](./images/f1-score.png)[reference](https://stackoverflow.com/questions/35365007/tensorflow-precision-recall-f1-score-and-confusion-matrix)

Is our F1-score surprising?

What do we think about the model?

Should we evaluate out-of-sample performance?

### Step 8b: Evaluate Performance on Out-of-Sample Data

In [53]:
y_valid = df_valid['compensated_delays']
X_valid = df_valid.drop(columns=['compensated_delays'])

In [54]:
y_pred_valid = est_model.predict(X_valid)
y_pred_valid[0:5]

array([0, 0, 1, 0, 1])

In [55]:
y_pred_valid_probs = pd.DataFrame(est_model.predict_proba(X_valid))
y_pred_valid_probs.head()

,0,1
0,0.675206,0.324794
1,0.821830,0.178170
2,0.441084,0.558916
3,0.686269,0.313731
4,0.392882,0.607118


Do you think we'll do better or worse or same?

#### Evaluate Performance via Confusion Matrix

In [56]:
confusion_matrix(y_true=y_valid,
                 y_pred=y_pred_valid)

array([[763758, 360329],
       [  5981,  13075]])

#### Evaluating Performance with AUC ROC

In [57]:
roc_auc_score(y_true=y_valid,
              y_score=y_pred_valid_probs.iloc[:, 1])

0.7644348618142321

#### Evaluating Performance via F1

In [58]:
f1_score(y_true=y_valid,
         y_pred=y_pred_valid)

0.06663099424145136

## Step 9: Determine Next Steps

What should the next model iteration focus on?